In [1]:
import os
%pwd

'/Users/whysocurious/Documents/MLDSAIProjects/e2e-mlops-gcp/research'

In [2]:
os.chdir("../")
%pwd

'/Users/whysocurious/Documents/MLDSAIProjects/e2e-mlops-gcp'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    top_n: int
    ml_uri: str
    hpo_exp: str
    exp_name: str
    rf_params: list
    

In [4]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            top_n=config.top_n,
            ml_uri=config.ml_uri,
            hpo_exp=config.hpo_exp,
            exp_name=config.exp_name,
            rf_params=config.rf_params,
            
        )

        return model_evaluation_config

In [7]:
import os
import mlflow
import pickle
import mlflow
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from mlProject import logger

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    def load_pickle(self, filename):
        with open(filename, "rb") as f_in:
            return pickle.load(f_in)


    def run_register_model(self):

        mlflow.set_tracking_uri(self.config.ml_uri)
        mlflow.set_experiment(self.config.exp_name)
        mlflow.sklearn.autolog()
        client = MlflowClient(tracking_uri=self.config.ml_uri)

        # X_train, y_train = self.load_pickle(os.path.join(self.config.data_path, "train.pkl"))
        X_val, y_val = self.load_pickle(os.path.join(self.config.data_path, "val.pkl"))
        X_test, y_test = self.load_pickle(os.path.join(self.config.data_path, "test.pkl"))

        # Retrieve the top_n model runs and log the models
        logger.info("Retrieve the top_n model runs and log the models.")
        experiment = client.get_experiment_by_name(self.config.hpo_exp)
        runs = client.search_runs(
            experiment_ids=experiment.experiment_id,
            run_view_type=ViewType.ACTIVE_ONLY,
            max_results=self.config.top_n,
            order_by=["metrics.rmse ASC"]
        )
        logger.info(len(runs))
        logger.info("logging top_n models wiht test metrics.")
        
        for run in runs:
            logger.info((str(run.info.run_id), str(run.data.metrics), str(run.data.params)))
            modelPath = client.download_artifacts(run_id=run.info.run_id, path="model")
            pipeLine = self.load_pickle(os.path.join(modelPath, "model.pkl"))

            with mlflow.start_run():

                mlflow.set_tag("model", "randomforest_topN")
                mlflow.log_params(run.data.params)
                
                logger.info("Evaluate model on the validation and test sets")
                val_rmse = mean_squared_error(y_val, pipeLine.predict(X_val), squared=False)
                mlflow.log_metric("val_rmse", val_rmse)
                test_rmse = mean_squared_error(y_test, pipeLine.predict(X_test), squared=False)
                mlflow.log_metric("test_rmse", test_rmse)
                mlflow.sklearn.log_model(pipeLine, artifact_path="model")

        logger.info("Selecting the model with the lowest test RMSE")
        experiment = client.get_experiment_by_name(self.config.exp_name)
        best_run = client.search_runs(
            experiment_ids=experiment.experiment_id,
            run_view_type=ViewType.ACTIVE_ONLY,
            max_results=self.config.top_n,
            order_by=["metrics.test_rmse ASC"]
        )[0]

        # Register the best model
        logger.info("Registering the best model")
        run_id = best_run.info.run_id
        model_uri = f"runs:/{run_id}/model"
        mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

In [8]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.run_register_model()
except Exception as e:
    raise e

[2024-07-05 15:14:57,085: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-05 15:14:57,088: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-05 15:14:57,089: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-05 15:14:57,090: INFO: common: created directory at: artifacts]
[2024-07-05 15:14:57,090: INFO: common: created directory at: artifacts/model_evaluation]


2024/07/05 15:14:57 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


[2024-07-05 15:14:59,198: INFO: 141086892: Retrieve the top_n model runs and log the models.]
[2024-07-05 15:14:59,266: INFO: 141086892: 2]
[2024-07-05 15:14:59,267: INFO: 141086892: logging top_n models wiht test metrics.]
[2024-07-05 15:14:59,267: INFO: 141086892: ('b62c83ca9ac647e9b34efc25183ef6de', "{'rmse': 5.073795325986875}", "{'max_depth': '14', 'min_samples_leaf': '3', 'min_samples_split': '4', 'n_estimators': '15', 'random_state': '42'}")]
[2024-07-05 15:14:59,340: INFO: 141086892: Evaluate model on the validation and test sets]


/Users/whysocurious/.local/share/virtualenvs/e2e-mlops-gcp-OPqJYq4S/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/whysocurious/.local/share/virtualenvs/e2e-mlops-gcp-OPqJYq4S/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


[2024-07-05 15:15:20,632: INFO: 141086892: ('5c87e742ae454a35b518bd45c26b0817', "{'rmse': 5.081163597340483}", "{'max_depth': '13', 'min_samples_leaf': '4', 'min_samples_split': '7', 'n_estimators': '25', 'random_state': '42'}")]
[2024-07-05 15:15:20,678: INFO: 141086892: Evaluate model on the validation and test sets]


/Users/whysocurious/.local/share/virtualenvs/e2e-mlops-gcp-OPqJYq4S/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/whysocurious/.local/share/virtualenvs/e2e-mlops-gcp-OPqJYq4S/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


[2024-07-05 15:15:45,827: INFO: 141086892: Selecting the model with the lowest test RMSE]
[2024-07-05 15:15:45,853: INFO: 141086892: Registering the best model]


Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2024/07/05 15:15:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.
